In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
with open('pickle/crime_borough_month_prev_year_values.pkl', 'rb') as picklefile:
    crime = pickle.load(picklefile)

# Non year based info | Borough level

In [3]:
borough_non_year_based = (pd
                          .read_excel('data/london-borough-atlas.xlsx',sheet_name='NonYearBased')
                          .rename(columns={'Area name':'borough'})
                          .drop('New code',axis=1))

In [4]:
# Selecting columns for borough non year based info

columns_to_keep = ['borough',"Simpson's Ethnic Diversity Index (16 ethnic groups)"]
# the rest of them are in the census information

borough_non_year_based = borough_non_year_based[columns_to_keep]

In [5]:
# with open('pickle/borough_non_year.pkl', 'wb') as picklefile:
#     pickle.dump(borough_non_year_based, picklefile)

# Year based info | Borough Level

In [6]:
year_based = pd.read_excel('data/london-borough-atlas.xlsx',sheet_name='YearBased')

#### Transposing the table wide to long with a new column with years

In [7]:
variable_names = year_based.iloc[0]
new_header = year_based.iloc[1]
year_based = year_based[2:] 
year_based.columns = new_header 

In [8]:
metainfo=[]
info = []
for i in range(2,len(variable_names)-1):
    if variable_names[i]==variable_names[i+1]:
        template = pd.DataFrame({'New code':year_based.iloc[:,0],'Area name':year_based.iloc[:,1]})
        template[variable_names[i]] = year_based.iloc[:,i]
        template['Year'] = year_based.columns[i]
        info.append(template)
    elif (variable_names[i]!=variable_names[i+1]):
        template = pd.DataFrame({'New code':year_based.iloc[:,0],'Area name':year_based.iloc[:,1]})
        template[variable_names[i]] = year_based.iloc[:,i]
        template['Year'] = year_based.columns[i]
        info.append(template)
        metainfo.append(info)
        info = []
template = pd.DataFrame({'New code':year_based.iloc[:,0],'Area name':year_based.iloc[:,1]})
template[variable_names[i]] = year_based.iloc[:,i+1]
template['Year'] = year_based.columns[i+1]
info.append(template)    
metainfo.append(info)

In [9]:
borough_year_based = pd.concat(metainfo[0])
for i in range(1,len(metainfo)):
    borough_year_based = (borough_year_based
                          .merge(pd
                                 .concat(metainfo[i]),on=['New code','Area name','Year'],how='outer'))

In [10]:
# Keeping only those years for which I've got crime data
borough_year_based = borough_year_based[borough_year_based['Year']>2006].drop(['New code'],axis=1)

In [11]:
# Creating a copy of the df with shifted years for future merge with crime data
# (Shifting +1 the year number to use the data as a lagged feature for the modeling)

shifted = borough_year_based[:]
shifted['Year'] = shifted['Year'] + 1

In [12]:
# Creating columns names with the name of the feature and the prefix 'prev'
new_cols = []

for i in shifted.columns[1:]:
    new_cols.append('Prev_year_'+i)

new_cols = ['borough'] + new_cols
shifted.columns = new_cols
shifted = shifted.rename(columns={'Prev_year_Year':'year'})

In [13]:
# Deleting columns with only one year of information
columns_to_keep = []
for col in shifted:
    if shifted[col].isna().sum()<72:
        columns_to_keep.append(col)
        
shifted = shifted[columns_to_keep]

In [14]:
for col in shifted.columns[1:]:
    shifted[col]= shifted[col].astype('float')

In [15]:
with open('pickle/borough_year.pkl', 'wb') as picklefile:
    pickle.dump(shifted, picklefile)

In [16]:
# Used the job_density to fill the fields in the borough file.
#job_density = pd.read_csv('https://files.datapress.com/london/dataset/jobs-and-job-density-borough/2018-02-27T13:57:27.18/Jobs%20density.csv')